In [ ]:
!pip install lyricsgenius

     |████████████████████████████████| 61kB 7.0MB/s 


In [ ]:
#libraries used to extract, clean and manipulate the data
#from helpers import *
import pandas as pd
import numpy as np
import string
#To plot the graphs
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')
#library used to count the frequency of words
from sklearn.feature_extraction.text import CountVectorizer
#To create the sentiment analysis model, tokenization and lemmatization
import nltk
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

In [ ]:
import lyricsgenius as genius
import pandas as pd
import string 

def search_data(query,n,access_token):
    """
    This function uses the library lyricsgenius to extract the fields
    title, artist, album, date and lyrics and stores them into a pandas dataframe
    parameters:
    query = artist or band to search
    n = max numbers of songs
    access_token = your access token of the genius api
    """
    
    api = genius.Genius(access_token)

    list_lyrics = []
    list_title = []
    list_artist = []
    list_album = []
    list_year = []

    artist = api.search_artist(query,max_songs=n,sort='popularity')
    songs = artist.songs
    for song in songs:
        list_lyrics.append(song.lyrics)
        list_title.append(song.title)
        list_artist.append(song.artist)
        #list_album.append(song.album)
        #list_year.append(song.year)

    df = pd.DataFrame({'artist':list_artist,'title':list_title,
                        'lyric':list_lyrics})
    
    return df

def clean_lyrics(df,column):
    """
    This function cleans the words without importance and fix the format of the  dataframe's column lyrics 
    parameters:
    df = dataframe
    column = name of the column to clean
    """
    df = df
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
    df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
    df[column] = df[column].str.replace("\n"," ").str.replace(r"[^\w\d'\s]+","").str.replace("efil ym fo flah","")
    df[column] = df[column].str.strip()

    return df



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
access_token = "eMiXYR0GF3D5dBmYyjqke6STCiR3MoWJ6sMZFwe4AG14vIwLvuZ8qRRzK157HKcd"
#list1 = [' Adele ', ' Joey + Rory ', ' Draaco Aventura ', ' Justin Bieber ', ' Chris Janson ', ' One Direction ', ' Drake ', ' Carrie Underwood ', ' Ed Sheeran ', ' Taylor Swift ', ' SayWeCanFly ', ' Selena Gomez ', ' Chris Brown ', ' Nicki Minaj ', ' Fifth Harmony ', ' Thomas Rhett  ', ' Eminem ', ' Beyonce ', ' Meghan Trainor ', ' Twenty One Pilots ', ' Fetty Wap ', ' 5 Seconds Of Summer ', ' Jason Aldean ', ' Luke Bryan ', ' Blake Shelton ', ' Rihanna ', ' Elvis Presley ', ' Ariana Grande ', ' JD Shelburne ', ' Lil Wayne ', ' Demi Lovato ', ' Maroon 5 ', ' Sam Hunt ', ' George Strait ', ' Wiz Khalifa ', ' Miranda Lambert ', ' DJ Khaled ', ' Porno for Pyros ', ' Katy Perry ', ' Miley Cyrus ', ' Tim McGraw ', ' The Weeknd ', ' Macklemore & Ryan Lewis ', ' Ellie Goulding ', ' Michael Jackson ', ' Rascal Flatts ', ' Britney Spears ', ' Zac Brown Band ', ' Hearty2Raw ', ' Future ', ' Lady Antebellum ', ' Adam Levine ', ' Janet Jackson ', ' Keith Urban ', ' Sam Smith ', ' Jason Derulo ', ' Tori Kelly ', ' 2Pac ', ' Ciara ', ' Old Dominion ', ' Randy Houser ', ' Eric Church ', ' Shakira ', ' Bruno Mars ', ' Little Big Town ', ' Rick Ross ', ' Toby Keith ', ' Brian Collins ', ' Sia ', ' Dixie Chicks ', ' Kanye West ', ' Shania Twain ', ' Enrique Iglesias ', ' The Band Perry ', ' Kellie Pickler ', ' Randy Owen ', ' Kelsea Ballerini ', ' Lana Del Rey ', ' Freddie Mercury ', ' Martin Garrix ', ' Brett Eldredge ', ' Trey Songz ', ' Young Thug ', ' Mariah Carey ', ' Jennifer Lopez ', ' Shawn Mendes ', ' Halsey ', ' The XX ', ' Kurt Stevens ', ' R. Kelly ', ' 50 Cent ', ' Kenny Chesney ', ' Linkin Park ', ' Akon ', ' Avenged Sevenfold ', ' John Legend ', ' Cole Swindell ', ' Justin Moore ', ' Jewel ', ' Maddie & Tae ', ' Chris Young ', ' Hunter Hayes ', ' Canaan Smith ', ' Madonna ', ' Albert Phillips ', ' Brantley Gilbert ', ' Coldplay ', ' Jay Z ', ' Lady Gaga ', ' Lil Fizz ', ' Iggy Azalea ', ' Chase Rice ', ' Danielle Bradbery ', ' Dustin Lynch ', ' Kacey Musgraves ', ' Alan Jackson ', ' Backstreet Boys ', ' Hank Williams, Jr. ', ' The Vamps ', ' Jaden Smith ', ' JoJo ', ' Kendrick Lamar ', ' Becky G ', ' Josh Turner ', ' Brad Paisley ', ' Gwen Stefani ', ' Mikel Knight ', ' Rachel Platten ', ' Reba McEntire ', ' Darius Rucker ', ' Florida Georgia Line ', ' Barbra Streisand ', ' Fall Out Boy ', ' Prince ', ' K. Michelle ', ' Little Mix ', ' BLACKBEAR ', ' Calvin Harris ', ' Jana Kramer ', ' Kenny Rogers ', ' T.I. ', ' Troye Sivan ', ' Brothers Osborne ', ' Rita Ora ', ' TLC ', ' Aaliyah ', ' Kirk Franklin ', ' Kid Rock ', ' Skrillex ', ' Ed Roman ', ' Don Williams ', ' Lonestar ', ' Nelly ', ' Willie Nelson ', ' Zonke ', ' Charli XCX ', ' X Ambassadors ', ' Jessie J ', ' Westlife ', ' Tinashe ', ' DMX ', ' Meek Mill ', ' Usher ', ' BRANDY CLARK ', ' Dolly Parton ', ' Foo Fighters ', ' Mark Wills ', ' Prince Royce ', ' Pentatonix ', ' Flo Rida ', ' Whitney Houston ', ' Avril Lavigne ', ' Pharrell Williams ', ' Dwight Yoakam ', ' Lee Brice ', ' Barbara Mandrell ', ' Black Veil Brides ', ' Jamey Johnson ', ' Jimmy Fortune ', ' YonnieMcfly ', ' Johnny Cash ', ' Nick Jonas ', ' Vince Gill ', ' Imagine Dragons ', ' Jake Owen ', ' Larry Hernandez ', ' Martina McBride ', ' Colt Ford ', ' Dierks Bentley ', " Guns N' Roses ", ' Justin Timberlake ', ' Kelly Clarkson ', ' Metallica ', ' Scotty McCreery ', ' The Weekend ', ' Bim Skala Bim ', ' Mark Ronson ', ' Merle Haggard ', ' Bea Miller ', ' Alessia Cara ', ' A Thousand Horses ', ' Brooks & Dunn ', ' Elle King ', ' Rod Stewart ', ' The Cadillac Three ', ' Jacob Whitesides ', ' Ja Rule ', ' Lauren Alaina ', ' Marc Anthony ', ' The Game ', ' The Pussycat Dolls ', ' Tyler Farr ', ' Clare Dunn ', ' Adam Lambert ', ' Andrea Bocelli ', ' Gary Allan ', ' George Jones ', ' Joe Jonas ', ' Krystal Keith ', ' Ray Charles ', ' Red Hot Chili Peppers ', ' The Oak Ridge Boys ', ' Kevin Gates ', ' Cassadee Pope ', ' August Alsina ', ' Rich Homie Quan ', ' Air Supply ', ' Brandy ', ' OneRepublic ', ' Pitbull ', ' Robin Thicke ', ' Snoop Dogg ', ' Travis Tritt ', ' Chvrches ', ' Lorde ', ' Alison Krauss & Union Station ', ' A$AP Rocky ', ' Jeremih ', ' Panic! At the Disco ', ' Sheryl Crow ', ' Trace Adkins ', ' Tyga ', ' Ruby Rose ', ' Bone Thugs-n-Harmony ', ' David Nail ', ' Duran Duran ', ' Kip Moore ', ' Wallpaper. ', ' Dej Loaf ']
#'Adele', 'Justin Bieber',' One Direction ','Halsey','Adam Levine','Jason Derulo','Drake ', 'Carrie Underwood'
list2 =['Carrie Underwood ', ' Ed Sheeran ', ' Taylor Swift ', ' SayWeCanFly ', ' Selena Gomez ', ' Chris Brown ', ' Nicki Minaj ', ' Fifth Harmony ', ' Thomas Rhett  ', ' Eminem ', ' Beyonce ', ' Meghan Trainor ', ' Twenty One Pilots ', ' Fetty Wap ', ' 5 Seconds Of Summer ',' Rihanna ', ' Elvis Presley ', ' Ariana Grande ',' Lil Wayne ', ' Demi Lovato ', ' Maroon 5 ','Blake Shelton',' Miranda Lambert ', ' DJ Khaled ', ' Porno for Pyros ', ' Katy Perry ', ' Miley Cyrus ', ' Tim McGraw ', ' The Weeknd ', ' Macklemore & Ryan Lewis ', ' Ellie Goulding ', ' Michael Jackson ', ' Rascal Flatts ', ' Britney Spears ',' Shakira ', ' Bruno Mars ','Sia', 'Kanye West','Enrique Iglesias','Future','50 Cent','Akon','Shawn Mendes', 'Jennifer Lopez']

df1 = []
df1 = pd.DataFrame(df1)
import time
df0 = search_data('Gavin DeGraw', 50, access_token)
df1 = df0.append(df1)


Searching for songs by Gavin DeGraw...

Song 1: "I Don’t Want to Be"
Song 2: "Not Over You"
Song 3: "Chariot"
Song 4: "She Sets the City on Fire"
Song 5: "In Love With a Girl"
Song 6: "Fire"
Song 7: "Sweeter"
Song 8: "Best I Ever Had"
Song 9: "You Got Me"
Song 10: "Making Love With The Radio On"
Song 11: "Soldier"
Song 12: "Something Worth Saving"
Song 13: "More Than Anyone"
Song 14: "Follow Through"
Song 15: "We Belong Together"
Song 16: "Kite Like Girl"
Song 17: "Belief"
Song 18: "Harder to Believe"
Song 19: "Say I Am"
Song 20: "Finest Hour"
Song 21: "Annalee"
Song 22: "You Make My Heart Sing Louder"
Song 23: "Make a Move"
Song 24: "(Nice to Meet You) Anyway"
Song 25: "How Lucky Can a Man Get"
Song 26: "Just Friends"
Song 27: "Meaning"
Song 28: "Everything Will Change"
Song 29: "Glass"
Song 30: "Free"
Song 31: "Stealing"
Song 32: "Cheated on Me"
Song 33: "Crush"
Song 34: "I’m Gonna Try"
Song 35: "Run Every Time"
Song 36: "I Have You to Thank"
Song 37: "She Holds a Key"
Song 38: "New 

In [ ]:
df0

,artist,title,lyric
0,Gavin DeGraw,I Don’t Want to Be,[Verse 1]\nI don't need to be anything other\n...
1,Gavin DeGraw,Not Over You,[Verse 1]\nDreams: that's where I have to go\n...
2,Gavin DeGraw,Chariot,[Verse 1]\nStaring at a maple leaf\nLeaning on...
3,Gavin DeGraw,She Sets the City on Fire,[Chorus]\nEverybody knows she's a perfect ten\...
4,Gavin DeGraw,In Love With a Girl,[Verse 1]\nSo many people gonna say that they ...
5,Gavin DeGraw,Fire,[Verse 1]\nOh if there's one thing to be taugh...
6,Gavin DeGraw,Sweeter,[Intro]\nThere you go\n\n[Verse 1]\nYou\nYou d...
7,Gavin DeGraw,Best I Ever Had,"[Verse 1]\nMelt Antarctica, savin' Africa\nI f..."
8,Gavin DeGraw,You Got Me,[Verse 1]\nWhen the fear takes you down\nWhen ...
9,Gavin DeGraw,Making Love With The Radio On,[Verse 1]\nI tried giving up\nWalked through q...


In [ ]:
df1 = pd.read_excel('Result_Songs.xlsx')
df1